In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 
    
#r "nuget:Microsoft.Data.Analysis,0.2.0"
#r "nuget:XPlot.Plotly.Interactive,4.0.2"

In [ ]:
using System;
using System.IO;
using System.Linq;
using System.Xml;

using Microsoft.Data.Analysis;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;

using XPlot.Plotly;

In [ ]:
// https://github.com/dotnet/interactive/blob/main/samples/notebooks/csharp/Samples/DataFrame-Getting%20Started.ipynb

using Microsoft.AspNetCore.Html;
Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [ ]:
string[] fileEntries = Directory.GetFiles( @".", "*.xml" );

In [ ]:
fileEntries

In [ ]:
// coverage.cobertura.xml
// https://docs.microsoft.com/ja-jp/dotnet/core/testing/unit-testing-code-coverage?tabs=windows#integrate-with-net-test

var xmlDoc = new XmlDocument();
xmlDoc.Load( @"./coverage.cobertura.xml" );

In [ ]:
var node = xmlDoc.SelectSingleNode( "coverage" );
if( node != null )
{
    var strLineRate     = node.Attributes["line-rate"].InnerText;
    var strLinesCovered = node.Attributes["lines-covered"].InnerText;
    var strLinesValid   = node.Attributes["lines-valid"].InnerText;

    var dLineRate       = double.Parse( strLineRate );
    var nLinesCovered   = int.Parse( strLinesCovered );
    var nLinesValid     = int.Parse( strLinesValid );

    Console.WriteLine( $"{dLineRate} {nLinesCovered}/{nLinesValid}" );
}


In [ ]:
DataFrame df = new DataFrame();
df.Columns.Add( new PrimitiveDataFrameColumn<DateTime>("datetime") );
df.Columns.Add( new PrimitiveDataFrameColumn<double>("line_rate") );
df.Columns.Add( new PrimitiveDataFrameColumn<int>("lines_covered") );
df.Columns.Add( new PrimitiveDataFrameColumn<int>("lines_valid") );

In [ ]:
df.Append( new List<object> { DateTime.Parse("2019/01/01"), 100, 12, 12 } );

In [ ]:
df

In [ ]:
var chart1 = Chart.Plot(
    new Scatter
    {
        x = df["datetime"],
        y = df["line_rate"],        
        mode = "lines+markers"
    }
);
var chart1_layout = new Layout.Layout{
    title="coverage",
    xaxis =new Xaxis{
        title = "Date",
        // https://plotly.com/javascript/reference/layout/xaxis/#layout-xaxis-tickformat
        // https://github.com/d3/d3-time-format/tree/v2.2.3#locale_format
        tickformat = "%Y/%m/%d"
    },
    yaxis = new Yaxis{
        title = "rate[%]",
        // https://plotly.com/javascript/reference/layout/xaxis/#layout-xaxis-range
        autorange = false,
        type = "linear",
        range = new int[] { 0, 100 }
    }           
};
chart1.WithLayout( chart1_layout );
chart1

In [ ]:
var htmlGraph = chart1.GetHtml();
System.IO.File.WriteAllText( "./graph.html", htmlGraph );